In [1]:
import keras
from keras import layers
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('wordnet')
import string
from nltk.corpus import stopwords
nltk.download('stopwords')
import os
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk.stem import SnowballStemmer,WordNetLemmatizer
stemmer=SnowballStemmer('english')
lemma=WordNetLemmatizer()
from string import punctuation
import re
import gensim
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Dense, Embedding, Flatten,Dropout
from keras.layers import SpatialDropout1D,GlobalMaxPooling1D
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.models import Sequential
from keras.models import Model
from sklearn.preprocessing import LabelBinarizer
from keras.layers import Embedding
from keras.models import load_model
from keras import backend as K

Using TensorFlow backend.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
embedding_dim = 300

In [3]:
url = "https://raw.githubusercontent.com/cacoderquan/Sentiment-Analysis-on-the-Rotten-Tomatoes-movie-review-dataset/master/train.tsv"
data = pd.read_csv(url,sep='\t')
data.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [0]:
def clean_review(review_col):
    review_corpus=[]
    for i in range(0,len(review_col)):
        review=str(review_col[i])
        review=re.sub('[^a-zA-Z]',' ',review)
        # tokens = word_tokenize(review)
        # tokens = [w.lower() for w in tokens]
        # table = str.maketrans('','',string.punctuation)
        # stripped = [w.translate(table) for w in tokens]
        # review = [word for word in review if word.isalpha()]
        # stop_words= set(stopwords.words('english'))
        # review = [w for w in review if not w in stop_words]
        review=[lemma.lemmatize(w) for w in word_tokenize(str(review).lower())]
        review=' '.join(review)
        review_corpus.append(review)
    return review_corpus

In [5]:
data['clean_review']=clean_review(data.Phrase.values)
data.head()

,PhraseId,SentenceId,Phrase,Sentiment,clean_review
0,1,1,A series of escapades demonstrating the adage ...,1,a series of escapade demonstrating the adage t...
1,2,1,A series of escapades demonstrating the adage ...,2,a series of escapade demonstrating the adage t...
2,3,1,A series,2,a series
3,4,1,A,2,a
4,5,1,series,2,series


In [6]:
phrase_line = list()
lines=data['Phrase'].values.tolist()
for line in lines:
  tokens = word_tokenize(line)
  tokens = [w.lower() for w in tokens]
  table = str.maketrans('','',string.punctuation)
  stripped = [w.translate(table) for w in tokens]
  words = [word for word in stripped if word.isalpha()]
  stop_words= set(stopwords.words('english'))
  words = [w for w in words if not w in stop_words]
  phrase_line.append(words)
len(phrase_line)

156060

In [7]:
word2Vec_model = gensim.models.Word2Vec(sentences=phrase_line,size=embedding_dim,window=5,workers=4,min_count=1)
words=list(word2Vec_model.wv.vocab)
print(len(words))

16063


In [8]:
filename='1100690_embedding_word2vec.txt'
word2Vec_model.wv.save_word2vec_format(filename,binary=False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
from keras.utils import to_categorical
X = data['clean_review']
Y = to_categorical(data['Sentiment'].values)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.30, random_state=1000)

In [11]:
all_words=' '.join(X_train)
all_words=word_tokenize(all_words)
#print(all_words)
dist=FreqDist(all_words)

num_unique_word=len(dist)
num_unique_word
#X_train.head()

13719

In [12]:
r_len=[]
for text in X_train:
    word=word_tokenize(text)
  #  print(text)
    l=len(word)
    r_len.append(l)
    
MAX_REVIEW_LEN=np.max(r_len)
MAX_REVIEW_LEN

48

In [0]:
max_features = num_unique_word
max_words = MAX_REVIEW_LEN
batch_size = 128
num_classes=5

In [14]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
# vocab_size = len(tokenizer.word_index) + 1

Found 13719 unique tokens.


In [16]:
X_train = pad_sequences(X_train, padding='post', maxlen=max_words)
X_test = pad_sequences(X_test, padding='post', maxlen=X_train.shape[1])
print('Shape of X train and X validation tensor:', X_train.shape,X_test.shape)
print('Shape of label train and validation tensor:', y_train.shape,y_test.shape)

Shape of X train and X validation tensor: (109242, 48) (46818, 48)
Shape of label train and validation tensor: (109242, 5) (46818, 5)


In [0]:
tokenizer_object = Tokenizer()
tokenizer_object.fit_on_texts(phrase_line)
word_index = tokenizer_object.word_index

In [18]:
embeddings_index = {}
f = open(os.path.join('', '/content/1100690_embedding_word2vec.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

Found 16064 word vectors.


In [19]:
# EMBEDDING_DIM=300
vocabulary_size=min(len(word_index)+1,max_features)
embedding_layer = Embedding(len(word_index) + 1,
                            embedding_dim,
                            weights=[embedding_matrix],
                            input_length=X_train.shape[1],
                            trainable=False)
# embedding_layer = Embedding(vocabulary_size,
#                             embedding_dim)

In [20]:
model = Sequential()
model.add(embedding_layer)
model.add(SpatialDropout1D(0.1))
model.add(layers.Conv1D(100, 3, activation='relu'))
model.add(layers.Conv1D(128, 3, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(Dense(5, activation='sigmoid'))
model.add(layers.Dense(num_classes, activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()










Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 48, 300)           4819200   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 48, 300)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 46, 100)           90100     
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 44, 128)           38528     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290 

In [0]:
history = model.fit(X_train, y_train,
                    epochs=100,
                    validation_data=(X_test, y_test),
                    batch_size=32)
model.save('1100690_1dconv_reg')

In [0]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [33]:
model = load_model('1100690_1dconv_reg')
loss_training, accuracy_training = model.evaluate(X_train, y_train, verbose=False)
loss,accuracy = model.evaluate(X_test, y_test, verbose=False)
y_pred = model.predict(X_test)
print("Training Accuracy: {:.4f}".format(accuracy_training))
print("Testing Accuracy:  {:.4f}".format(accuracy))
print("Testing f1_score:  {:.4f}".format(K.get_value(K.to_dense(f1_m(y_test,y_pred)))))
print("Testing precision:  {:.4f}".format(K.get_value(K.to_dense(precision_m(y_test,y_pred)))))
print("Testing recall:  {:.4f}".format(K.get_value(K.to_dense(recall_m(y_test,y_pred)))))

Training Accuracy: 0.6829
Testing Accuracy:  0.5858
Testing f1_score:  0.5647
Testing precision:  0.6208
Testing recall:  0.5179
